Cell 1: Importing Libraries and Setting API Key

In [ ]:
# Importing Libraries
import requests
import base64
import pandas as pd

# Your Companies House API key
API_KEY = ''

Cell 2: Defining Functions

In [ ]:
# Function for Searching Director and Secretary
def get_company_details(company_name, base64_api_key):
    # The endpoint URL for searching companies
    search_endpoint = 'https://api.companieshouse.gov.uk/search/companies'
    
    # Set up the parameters for the search query
    params = {'q': company_name}
    
    # Set up the request headers with the encoded API key
    headers = {'Authorization': 'Basic ' + base64_api_key}
    
    # Send a GET request to search for the company by name
    response = requests.get(search_endpoint, params=params, headers=headers)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the search results in JSON format
        search_results = response.json()
        
        # Check if search results exist
        if search_results['items']:
            # Filter out dissolved companies
            active_companies = [item for item in search_results['items'] if item.get('company_status', '') != 'dissolved']
            if active_companies:
                # Extract details from the first active company
                company_number = active_companies[0]['company_number']
                company_address = active_companies[0]['address_snippet']
                return company_number, company_address
            else:
                # If no active companies are found, print a message
                print(f'No active company found matching the name: {company_name}')
                return None, None
        else:
            # If no companies are found, print a message
            print(f'Company not found: {company_name}')
            return None, None
    else:
        # If there's an error in the request, print the status code
        print(f'Error searching companies for {company_name}:', response.status_code)
        return None, None

def get_director_and_secretary_details(company_number, base64_api_key):
    # Endpoint URL for fetching officer details of a company
    officers_endpoint = f'https://api.companieshouse.gov.uk/company/{company_number}/officers'
    
    # Set up the request headers with the encoded API key
    headers = {'Authorization': 'Basic ' + base64_api_key}
    
    # Send a GET request to retrieve officer details
    response = requests.get(officers_endpoint, headers=headers)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the officer data in JSON format
        officers_data = response.json().get('items', [])
        
        # Initialize variables to hold director and secretary details
        director_name, director_role = 'N/A', 'N/A'
        secretary_name, secretary_role = 'N/A', 'N/A'
        
        # Loop through officer data to find director and secretary
        for officer in officers_data:
            if officer.get('resigned_on') is None:
                if officer.get('officer_role').lower() == 'director':
                    director_name = officer.get('name', 'N/A')
                    director_role = officer.get('officer_role', 'N/A')
                elif officer.get('officer_role').lower() == 'secretary':
                    secretary_name = officer.get('name', 'N/A')
                    secretary_role = officer.get('officer_role', 'N/A')
        
        # Return director and secretary details as a dictionary
        return {
            'Director Name': director_name,
            'Director Role': director_role,
            'Secretary Name': secretary_name,
            'Secretary Role': secretary_role
        }
    
    else:
        # If there's an error in the request, print the status code
        print(f'Error retrieving officers for company number {company_number}:', response.status_code)
        return None


Cell 3: List of Company Names

In [ ]:
# This cell is used to encode the API key and define company names list
base64_api_key = base64.b64encode(API_KEY.encode()).decode()

# Example list of company names
company_names = []


In [ ]:
# Extract Company Names from an Excel File
# Path to the Excel file (you need to specify this)
excel_file_path = ''

# Specify the sheet name or index and column name
sheet_name = 'Sheet1'  # or the index, e.g., 0 for the first sheet
column_name = 'Company name'  # Replace 'Column_Name' with the name of your column

# Load the Excel file
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Ensure the column exists to avoid KeyError
if column_name in df.columns:
    # Add the data from the column to your list
    company_names = df[column_name].tolist()
    print("Data added to the list successfully!")
else:
    print(f"The column '{column_name}' does not exist in the sheet '{sheet_name}'.")

# Print the updated list (optional)
print(company_names)

Cell 4: Extracting and Processing Data

In [ ]:
# Extracting Data from Companies House
    # Prepare the list to hold all the rows
table_rows = []

    # Iterate through each company name in the list
for company_name in company_names:
    try:
        company_number, company_address = get_company_details(company_name, base64_api_key)
        if company_number and company_address:
            # Fetch director and secretary details
            director_secretary_data = get_director_and_secretary_details(company_number, base64_api_key)
                
            if director_secretary_data:
                # Append row with company details, director details, and secretary details
                table_rows.append([
                    company_name, company_number, company_address,
                    director_secretary_data['Director Name'], director_secretary_data['Director Role'],
                    director_secretary_data['Secretary Name'], director_secretary_data['Secretary Role']
                ])
                print(f"Successfully retrieved data for company: {company_name}")
            else:
                print(f"No director and secretary data found for company: {company_name}")
        else:
            print(f"No valid company details found for company: {company_name}")
    except Exception as e:
        print(f"Error retrieving data for company {company_name}: {e}")

Cell 5: Creating and Displaying DataFrame

In [ ]:
# Create a DataFrame from the table_rows
df = pd.DataFrame(table_rows, columns=['Company Name', 'Company Number', 'Company Address',
                                           'Director Name', 'Director Role',
                                           'Secretary Name', 'Secretary Role'])

# Display the DataFrame in Jupyter Notebook
df.tail()

Cell 6: Extracting Data to an Excel File

In [ ]:
# Export the DataFrame to an Excel file
df.to_excel('', index=False)

